In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

## Check the path of python module

In [4]:
pyspark.__file__

'/home/4oceanknowledges/spark/spark-3.4.2-bin-hadoop3/python/pyspark/__init__.py'

In [3]:
pd.__file__

'/home/4oceanknowledges/anaconda3/lib/python3.11/site-packages/pandas/__init__.py'

## Create Spark Session

In [5]:
# create spark session 

spark = SparkSession.builder\
    .master("local[*]")\
    .appName('test')\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/22 08:07:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Download and Extract Data

In [50]:
# download .csv data from github repo of DataTalksClub
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz
    

--2024-02-22 06:41:23--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 52.216.114.211, 54.231.140.41, 54.231.235.105, ...
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|52.216.114.211|:443... connected.
HTTP request sent, awaiting response... 403 Forbidden
2024-02-22 06:41:24 ERROR 403: Forbidden.



In [ ]:
!tar xzfv fhvhv_tripdata_2021-01.csv.gz

## Read data without inferring schema

In [6]:
# read the csv file as dataset in spark 
ds = spark.read\
    .option("header","true")\
    .csv('fhvhv_tripdata_2021-01.csv')

In [7]:
# show the dataset
ds.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [8]:
ds.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:33:44', dropoff_datetime='2021-01-01 00:49:07', PULocationID='230', DOLocationID='166', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime='2021-01-01 00:55:19', dropoff_datetime='2021-01-01 01:18:21', PULocationID='152', DOLocationID='167', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:23:56', dropoff_datetime='2021-01-01 00:38:05', PULocationID='233', DOLocationID='142', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:42:51', dropoff_datetime='2021-01-01 00:45:50', PULocationID='142', DOLocationID='143', SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime='2021-01-01 00:48:14', dropoff_datetime='2021-01-01 01:08:42', PULocationID='143', DOLocationID='78', SR_Flag=None)]

In [9]:
# check dataset schema 
ds.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

You can probably notice the schema is all in string type as spark do not explicitly define the datatype of the column 

## Taking part of the data as example for inferring schema 

In [10]:
# using pandas, we can easily know the datatype for each column
# to achieve this, we first put first 100 rows in a .csv file, as pandas read in not so big chunk of data

! head  -n 10001 fhvhv_tripdata_2021-01.csv > head.csv

In [11]:
!head -n 10 head.csv

In [10]:
# we can read it directly from spark using inferScehema 
spark_df = spark.read.csv("head.csv", header=True, inferSchema=True)


In [11]:
# schema 
spark_df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('SR_Flag', StringType(), True)])

## Create spark dataframe from pandas dataframe

In [13]:
# we can also create spark dataframe from pandas

spark_df = spark.createDataFrame(pandas_df)

In [12]:
# read data using pandas
pandas_df = pd.read_csv('head.csv')
pandas_df.head()

,hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
0,HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,NaN
1,HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,NaN
2,HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,NaN
3,HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,NaN
4,HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,NaN


In [14]:
spark_df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

From the inferred schema shown here, LongType() might not the good datatype compared to IntegerType() and the previos took up 8 bytes while latter took up 4 bytes. We might need to enforce a custom schema.

## Enforce custom schema 

In [16]:
from pyspark.sql import types

In [18]:
schema = types.StructType([
  types.StructField('hvfhs_license_num', types.StringType(), True), 
  types.StructField('dispatching_base_num', types.StringType(), True), 
  types.StructField('pickup_datetime', types.TimestampType(), True), 
  types.StructField('dropoff_datetime', types.TimestampType(), True), 
  types.StructField('PULocationID', types.IntegerType(), True), 
  types.StructField('DOLocationID', types.IntegerType(), True), 
  types.StructField('SR_Flag', types.StringType(), True)
])

In [42]:
ds = spark.read\
    .option("header","true")\
    .schema(schema)\
    .csv("fhvhv_tripdata_2021-01.csv")

In [43]:
ds.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

## Save file into parquet with multiple partition

In [45]:
ds = ds.repartition(numPartitions=24)

In [46]:
ds.write.parquet('fhvhv/2021/01',mode='overwrite')

## Spark DataFrame in details

In [47]:
df = spark.read.parquet('fhvhv/2021/01/')


In [48]:
# print the schema 
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [ ]:
df